In [ ]:
import json
import paho.mqtt.client as mqtt
mqtt_broker_port = 1883
mqtt_broker_host = (
    "fastapi-ecs-dev-mqtt-nlb-6ebce59-73957a14f2058112.elb.ap-northeast-1.amazonaws.com"
)
# Load the location list
with open("api/migrations/location_list.json", "r") as f:
    locations = json.load(f)

# Create an MQTT client
client = mqtt.Client()

# Connect to the MQTT broker
client.connect(mqtt_broker_host, mqtt_broker_port)

# For each 50 locations, publish a message
for i in range(0, len(locations), 100):
    message = locations[i : i + 100]
    client.publish("source_files", json.dumps(message))

# Disconnect from the MQTT broker
client.disconnect()

In [ ]:
# The callback for when the client receives a CONNACK response from the server.
import paho.mqtt.client as mqtt
import time

mqtt_broker_port = 1883
mqtt_broker_host = (
    "fastapi-ecs-dev-mqtt-nlb-6ebce59-73957a14f2058112.elb.ap-northeast-1.amazonaws.com"
)

message_count = 0
start_time = time.time()

def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe("weather/processed/#")


def on_message(client, userdata, msg):
    global message_count
    message_count += 1
    elapsed_time = time.time() - start_time
    messages_per_second = message_count / elapsed_time
    print(
        f"Messages per second: {messages_per_second}  "
    )
client = mqtt.Client()

client.connect(mqtt_broker_host, mqtt_broker_port)
client.on_connect = on_connect
client.on_message = on_message

client.loop_forever()